## This notebook inputs the missing values using KNN

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.impute import KNNImputer
from sklearn import preprocessing

#### Choose wether the imputation should be done on the data set with daily or weekly granularity

In [2]:
INPUT, OUTPUT = 'output/\CompleteWeeklyIndexes', 'output/\KNNWeeklyImputed'

In [3]:
df = pd.read_csv(INPUT+'.csv', parse_dates=True, index_col=0)
df_test = df.loc['2015-01-02':] #Selecting test data
df = df.loc['2000-01-01':'2015-01-01'] #Selecting training data

In [4]:
df_test.tail()

,sp500_Close,sp500_Volume,vix_Close,vix_Volume,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n225_Close,n225_Volume,...,hsi_Close,hsi_Volume,n100_Close,n100_Volume,Overall EMV Tracker,infectious_daily_infect_emv_index,GPR,trade_US Trade Policy Uncertainty,trade_Japanese Trade Policy Uncertainty,trade_Trade Policy EMV Fraction
2020-02-21,3366.854980,3.813365e+09,15.462500,0.0,29198.152344,2.740575e+08,9605.762695,2.553652e+09,23396.726172,60100.000000,...,27612.716016,1.566489e+09,1173.309985,208087340.0,NaN,4.341429,NaN,NaN,NaN,NaN
2020-02-28,3080.693946,6.307054e+09,31.942000,0.0,26635.150000,6.038540e+08,8737.351953,3.147614e+09,22030.697754,114950.000000,...,26663.830078,1.854099e+09,1080.093994,449170340.0,24.443546,29.937143,76.532662,NaN,NaN,NaN
2020-03-06,3044.006054,5.979102e+09,36.757999,0.0,26339.529688,5.638040e+08,8724.750195,4.040054e+09,21121.148047,104200.000000,...,26342.621875,2.292851e+09,1030.441992,434007680.0,NaN,26.794286,NaN,NaN,NaN,NaN
2020-03-13,2712.365967,8.104234e+09,57.792001,0.0,23361.727734,7.641180e+08,7917.065820,4.597718e+09,18994.524219,172440.000000,...,24801.312110,3.955053e+09,860.319995,722226640.0,NaN,57.690000,NaN,NaN,NaN,NaN
2020-03-20,2437.806641,8.298607e+09,78.350001,0.0,20441.606771,8.115167e+08,7223.170085,4.795060e+09,16913.373047,178033.333333,...,23163.650391,3.852502e+09,753.783346,627243300.0,NaN,52.960000,NaN,NaN,NaN,NaN


In [5]:
for i in range(0,30):
    #Scaled data has zero mean and unit variance
    scaler = preprocessing.StandardScaler()
    scaler.fit(df)
    array = scaler.transform(df)
    array_test = scaler.transform(df_test)
    df_scaled = pd.DataFrame(data=array, index=df.index, columns=df.columns)
    df_test_scaled = pd.DataFrame(data=array_test, index=df_test.index, columns=df_test.columns)
    
    imp = KNNImputer(n_neighbors=5, weights="uniform")
    imp.fit(df_scaled)
    df_imputed = imp.transform(df_scaled)
    df_test_imputed = imp.transform(df_test_scaled)
    df_final = pd.DataFrame(data=df_imputed, index=df.index, columns=df.columns)
    df_test_final = pd.DataFrame(data=df_test_imputed, index=df_test.index, columns=df_test.columns)
    # Descaling the data
    array_2 = scaler.inverse_transform(df_final)
    df_final2 = pd.DataFrame(data=array_2, index=df.index, columns=df.columns)

    array_test_2 = scaler.inverse_transform(df_test_final)
    df_test_final2 = pd.DataFrame(data=array_test_2, index=df_test.index, columns=df_test.columns)
    df_final2.to_csv(OUTPUT+"_"+str(i)+'_training.csv', index = True)
    df_test_final2.to_csv(OUTPUT+"_"+str(i)+'_test.csv', index = True)